# Import Functions

In [ ]:
from Models.Should._utils import *
from Models.Should.Dataset_creation import *
from Models.Should.Should_LSTM import *

# Before Running

In [ ]:
dataset_path = '/kaggle/input/should-azimuth-complete-simulations/Must_Should_processed' # ADD DATASET PATH
test_files = 'Trained_Models/test_files_Mxb1.txt' # ADD PATH TO FILES IN TEST SET OF PRETRAINED MODELS
model_path = 'Trained_Models/model_2025-01-24_Mxb1.pt' # ADD PATH TO PRETRAINED MODEL
load_axis = 'Mxb1' # MAKE SURE SELECTED MODEL MATCHES LOAD AXIS

# Create Dataset

In [ ]:
# Read the file containing the files in the test set
with open(test_files, 'r') as file:
    file_list = [line.strip('\n') for line in file]

# Select the corresponding label means and standard deviation of the model
label_means = {
    'Mxb1': (-1942.5360379140843, 21097.365971150757),
    'Myb1': (0, 0)
}
train_label_mean, train_label_std = label_means[load_axis]

# Create the dataset and dataloader
datasets = {
    'test': Should_Dataset(dataset_path, file_list, load_axis, train_label_mean, train_label_std),
}

batch_size = 4

dataloaders = {
    'test': DataLoader(datasets['test'], batch_size, shuffle=True)
}

print(f'The length of the test dataset is: {len(datasets["test"])}')

# Load the Model

In [ ]:
# Create the model
input_size = 2
hidden_size = 128
num_layers = 3
dropout = 0.2
proj_size = 1
model = Should_model(input_size=input_size, 
                    hidden_size=hidden_size, 
                    num_layers=num_layers, 
                    dropout=dropout,
                    proj_size=proj_size,
                    batch_size=batch_size
                    )


# Define the loss function and optimizer
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'The specified device is: {device}')
print(f'The model architecture is:\n{model}')

In [ ]:
# Load the specified model
model.load_state_dict(torch.load(model_path, weights_only=True))

# Perform Inference

In [ ]:
# Evaluate the model and plot inference
test_loss, test_del_error = evaluate(model, dataloaders['test'], loss_fn, device=device)
print(f'The loss over the test set is: {test_loss}\nThe test DEL error is: {test_del_error}')
plot_inference(model, dataloaders['test'], num_inf=3, num_splits=3, device=device)